In [ ]:

from itertools import islice
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
from torch.optim import SGD
import torchvision.transforms as T
from torchvision.datasets import MNIST
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from timeit import default_timer as timer
from torch.utils.data import Subset
import os
import sys
import random
import copy
from torch.backends import cudnn
import pandas as pd

# associo cuda per lavorare ulla gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
                #####################################################################################
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


                #####################################################################################


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):

  def __init__(self,numFC2=84):
      super(Net, self).__init__()
      self.numFC2 = numFC2
      self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
      self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
      self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
      self.conv5 = nn.Conv2d(128, 128, 3, padding=1)
      self.conv6 = nn.Conv2d(128, 256, 3, padding=1)
      self.fc1 = nn.Linear(256*8*8, 120)
      self.fc2 = nn.Linear(120, numFC2)
      self.task_fcs = []       # Will hold all Linear layers for classification heads.
      self.current_tasks = []  # Selects which task(s) are currently active.
      self.dropout = nn.Dropout(0.5)


  def forward(self, x):
    x =(F.relu(self.conv1(x)))
    x =(F.relu(self.conv2(x)))
    x =(F.relu(self.conv3(x)))
    x = self.pool(x)
    x =(F.relu(self.conv4(x)))
    x =(F.relu(self.conv5(x)))
    x =(F.relu(self.conv6(x)))
    x = self.pool(x)
    x = x.view(-1, 256*8*8)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout (F.relu(self.fc2(x)))
    # Concatenates the classification heads of selected tasks.
    #outputs = torch.cat([F.log_softmax(self.task_fcs[t](x)) for t in self.current_tasks], 1)
    outputs = torch.cat([(self.task_fcs[t](x)) for t in self.current_tasks], 1)
    return outputs


  # Add a new classification head with num_classes outputs.
  def add_task(self, num_classes):
      fc = nn.Linear(self.numFC2, num_classes)
      self.add_module(name=f'task{len(self.task_fcs)}_fc', module=fc)
      #self.add_module(name=str(len(self.task_fcs)), module=fc)
      self.task_fcs.append(fc)

  # Set the current task(s) -- takes a *LIST* of task ids.
  def set_tasks(self, tasks):
      self.current_tasks = tasks


                                         ###########################################################################

# CLASSE IL DATASET FILTRATO
class Filtered_Dataset(Dataset): 
  def __init__(self, dataset, indices, offset=0):
        self.indices = indices # indici dei target di cui ci interessa il target
        self.original_indices = [i for i in range(len(dataset.targets)) if dataset.targets[i] in indices] # filtraggio degli indici del dataset che voglio nel subset
        self.dataset = Subset(dataset, self.original_indices)
        #self.original2task = { indices[i]  : offset + i for i in range(0,  len(indices) ) } # remapping degli indici con chiavi i valori degli indici del del task
        self.original2task = self.dictmap(indices,offset)
        self.task2original = dict({ (val, key) for (key, val) in self.original2task.items() }) # remapping degli indici con chiavi i valori degli indici del del task
        
  def __getitem__(self, idx):
        (x, y) = self.dataset[idx]
        return (x, self.original2task[y])

  def __len__(self):
        return len(self.original_indices)

  def dictmap(self,indices,offset):
      original2task={}
      idx = np.arange(len(indices))
      idx_tmp = np.arange(len(indices))
      for i in range(0,len(idx)):
        if (idx[i]+offset) in indices:
          original2task[idx[i]+offset] = (idx[i] + offset)
          indices  = np.delete(indices, np.argwhere(indices == (idx[i]+offset)))
          idx_tmp  = np.delete(idx_tmp,np.argwhere(idx_tmp == (idx[i])))     
      for i in range(0,len(indices)):
          original2task[indices[i]]=idx_tmp[i] + offset   
      return original2task


                                          ###########################################################################

# METODO PER SPLITTARE GLI INDICI DEI TARGET DEL TRAINSET/TESTSET
def idx_tasks(num_tasks,trainset,rand=True):   # genero un np array e splitto in task
  targets = np.unique(trainset.targets)
  tasks = []
  if rand:
    unique_targets = np.random.permutation(len(targets))
    tasks = np.split(unique_targets,num_tasks,axis=0)
  else:  
    tasks = np.split(targets,num_tasks,axis=0)
  if num_tasks!=1 and rand:  
    for i in range(0,len(tasks)):
      tasks[i] = np.sort(tasks[i])
  return tasks # ritorno gli indici per poi usare nella funzione successiva

                                   #####################################################################################

# METODO CHE ESEGUE IL TRAIN DELLA RETE                                   
def train(trainloader,optimizer,criterion,scheduler,net):
    losses = []
    for epoch in range(20):  # loop over the dataset multiple times
        start = timer()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            labels = data[1]
            inputs, labels = data[0].to(device), labels.to(device) 
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            #print(outputs.dtype)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            loss.backward()
            optimizer.step() 
        mean_losses= sum(losses)/len(losses)
        #scheduler.step(mean_losses)
        end = timer()
        print("Tempo dell'epoca ",epoch,"uguale a: ", (end - start), "sec")
    print('ALLENAMENTO FINITO')
    print("################################") 

cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
#device = "cpu"
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

ds_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
ds_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                    download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
                    # NUOVI METODI IMPLEMENTATI: TEST GENERALE DELLA NET, TEST PER OGNI CLASSE, BACKBONE ROUTINE

# METODO PER GENERARE I FILTERED DATASET CON L'OFFSET
def task_dss(tasks,ds,offset_change=True): # aggiunto check su offset in modo tale da non cambaire offset ai task
    task_dss = []
    offset = 0
    for tsk in tasks:
      print(offset)
      tmp = Filtered_Dataset(ds, tsk, offset)
      task_dss.append(tmp)
      if offset_change:
          offset += len(tsk)
      else :
        offset = 0    
    return task_dss  
# METODO TASK SULLA RETE GENERALE    
def test(testloader,net,task_dss,num_task):
    # accuracy dell'intero network
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1) # torna il valore max della tupla
            if num_task <= 1:
              for keys in task_dss.task2original:
                  predicted[predicted == keys]=task_dss.task2original[keys]
                  labels[labels == keys]=task_dss.task2original[keys]  
            else:
              for task in task_dss:
                for keys in task.task2original:
                  predicted[predicted == keys]=task.task2original[keys]
                  labels[labels == keys]=task.task2original[keys]         
            total += labels.size(0)
            #print(total)
            correct += (predicted == labels).sum().item()
            #print('corrette',correct)
    net.train()
    print('Accuracy of the network : %d %%' % (
            100 * correct / total)) 
    return (100 * correct / total)

# METODO PER CALCOLO ACCURACY DELE CLASSI    
def test_class(testloader,net,task_dss,index,num_task):
    class_correct = list(0. for i in range(10)) 
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            
            _, predicted = torch.max(outputs, 1)
            if num_task <= 1:
              for keys in task_dss.task2original:
                  predicted[predicted == keys]=task_dss.task2original[keys]
                  labels[labels == keys]=task_dss.task2original[keys]  
            else:
              for task in task_dss:
                for keys in task.task2original:
                  predicted[predicted == keys]=task.task2original[keys]
                  labels[labels == keys]=task.task2original[keys]    
            c = (predicted == labels).squeeze()
            for i in range(16):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                #print('valore class total ',class_total[label])
    if num_task<=1:
        for i in index:
         print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))  
    else:
      for tsk in index:
        for i in tsk:
          print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))    
    print("################################")      

# METODO PER ESEGUIRE IL CONTIUAL LEARNING TASK AGNOSTIC

def continual_agnostic(net, tasks,task_train_dls,task_test_dls,task_test_dss): 
  tsk = [] # indici dei task incrementali 
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
  #optimizer = optim.Adam(net.parameters(), lr=0.0005)
  criterion = nn.CrossEntropyLoss()
  for i in range(0,len(tasks)):
    print(net)
    tsk.append(i)
    net.add_task(len(tasks[i]))
    net.set_tasks(tsk)
    net.to(device)
    net.train()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    #optimizer = optim.Adam(net.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss()
    train(task_train_dls[i],optimizer,criterion,net)
    if len(tsk)>1:
      #net.set_tasks(tsk) # TASK AGNOSTIC, non so quale task sto eseguendo, output solo
      #net.to(device)
      for i in range (0,len(tsk)):
        print(f'AFTER TRAINING TASKS : {tsk}')
        print('TASK ',i)
        acc =test(task_test_dls[i],net,task_test_dss[i],num_task=1)
    else:    
        net.set_tasks(tsk)
        net.to(device)
        acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
  return net    

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

ds_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                    download=True, transform=transform)
ds_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                    download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#METODO CHE PRINTA LE ACCURACY PRIMA E DOPO L'ULTIMO TASK CON TIPOLOGIA TASK_AGNOSTIC
def continual_table_agnostic(tasks,task_train_dls,task_test_dls,task_test_dss): 
  tsk = [] # indici dei task incrementali
  accuracies1 = []
  accuracies2 = [] 
  net = Net()
  for i in range(0,len(tasks)):
    index = i
    print(net)
    tsk.append(i)
    net.add_task(len(tasks[i]))
    net.set_tasks(tsk)
    net.to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.0005)
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4,verbose=True)
    train(task_train_dls[i],optimizer,criterion,scheduler,net)
    print('TASK ',i)
    acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
    accuracies1.append(acc)
    if  index == (len(tasks)-1):
      #net.set_tasks(tsk)
      #net.to(device)
      print(tsk)
      for idx in range (0,len(tsk)):
        acc_tmp = test(task_test_dls[idx],net,task_test_dss[idx],num_task=1)
        accuracies2.append(acc_tmp)
  df = pd.DataFrame()
  df['first_training']=accuracies1
  df['last_training']=accuracies2 
  avg1 = np.average(df['first_training'])
  avg2 = np.average(df['last_training']) 
  print("Accuracy Precedente:",avg1,"Accuracy Successiva:",avg2)
  forgetting= avg1-avg2
  print("Forgetting:",forgetting)         
  return net,df,forgetting



# METODO CHE PRINTA LE ACCURACY PRIMA E DOPO L'ULTIMO TASK CON TIPOLOGIA TASK_AWARE
def continual_table_aware(tasks,task_train_dls,task_test_dls,task_test_dss): 
  tsk = [] # indici dei task incrementali
  accuracies1 = []
  accuracies2 = [] 
  net = Net()
  for i in range(0,len(tasks)):
    index = i
    print(net)
    tsk.append(i)
    net.add_task(len(tasks[i]))
    net.set_tasks([i])
    net.to(device)
    for child  in trained_net_agnostic.children():
        last_layer = child
    #optimizer = optim.SGD(last_layer.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(last_layer.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4,verbose=True)
    train(task_train_dls[i],optimizer,criterion,scheduler,net)
    print('TASK ',i)
    net.set_tasks([i])
    net.to(device)
    acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
    accuracies1.append(acc)
    if  index == (len(tasks)-1):
      for idx in range (0,len(tsk)):
        print([idx])
        net.set_tasks([idx])
        net.to(device)
        acc_tmp = test(task_test_dls[idx],net,task_test_dss[idx],num_task=1)
        accuracies2.append(acc_tmp)
  df = pd.DataFrame()
  df['first_training']=accuracies1
  df['last_training']=accuracies2   
  avg1 = np.average(df['first_training'])
  avg2 = np.average(df['last_training']) 
  print("Accuracy Precedente:",avg1,"Accuracy Successiva:",avg2)
  forgetting= avg1-avg2
  print("Forgetting:",forgetting)   
  return net,df,forgetting  

In [ ]:
task_join = idx_tasks(1,ds_train,rand=False)
task_train_dss = task_dss(task_join, ds_train,offset_change=True) 
task_test_dss = task_dss(task_join, ds_test,offset_change=True) 
task_train_dls = [DataLoader(ds,batch_size=16,shuffle=True,num_workers=4) for ds in task_train_dss]
task_test_dls = [DataLoader(ds, batch_size=16,shuffle=False,num_workers=4) for ds in task_test_dss]

# JOINT-TRAIN
print(task_join)
trained_net_join,df_join,forgetting = continual_table_agnostic(task_join,task_train_dls,task_test_dls,task_test_dss)
print(df_join)

In [ ]:
def continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=True,agnsotic_test=True): 
  tsk = [] # indici dei task incrementali
  accuracies1 = []
  accuracies2 = [] 
  net = Net()
  for i in range(0,len(tasks)):
    index = i
    print(net)
    tsk.append(i)
    net.add_task(len(tasks[i]))
    if agnostic_train:
       net.set_tasks(tsk)
       net.to(device)
       optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
       #optimizer = optim.Adam(net.parameters(), lr=0.0005)
       criterion = nn.CrossEntropyLoss()
       scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4,verbose=True)
       train(task_train_dls[i],optimizer,criterion,scheduler,net)
       if agnsotic_test:
         acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
         accuracies1.append(acc)
       else:
           net.set_tasks([i])  
           net.to(device)  
           acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
           accuracies1.append(acc)
    else:
      net.set_tasks([i])  
      net.to(device) 
      optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
      #optimizer = optim.Adam(net.parameters(), lr=0.0005)
      criterion = nn.CrossEntropyLoss()
      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4,verbose=True)
      train(task_train_dls[i],optimizer,criterion,scheduler,net)
      if agnsotic_test:
          net.set_tasks(tsk)
          net.to(device)
          acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
          accuracies1.append(acc)
      else:
          net.set_tasks([i])  
          net.to(device)  
          acc = test(task_test_dls[i],net,task_test_dss[i],num_task=1)
          accuracies1.append(acc)
    print('TASK ',i)
    if  index == (len(tasks)-1):
      if agnsotic_test:
          net.set_tasks(tsk)
          net.to(device)   
          print(tsk)
          for idx in range (0,len(tsk)):
            acc_tmp = test(task_test_dls[idx],net,task_test_dss[idx],num_task=1)
            accuracies2.append(acc_tmp)
      else:
          print(tsk)
          for idx in range (0,len(tsk)):
            net.set_tasks([idx])
            net.to(device)   
            acc_tmp = test(task_test_dls[idx],net,task_test_dss[idx],num_task=1)
            accuracies2.append(acc_tmp)
  df = pd.DataFrame()
  df['first_training']=accuracies1
  df['last_training']=accuracies2 
  avg1 = np.average(df['first_training'])
  avg2 = np.average(df['last_training']) 
  print("Accuracy Precedente:",avg1,"Accuracy Successiva:",avg2)
  forgetting= avg1-avg2
  print("Forgetting:",forgetting)         
  return net,df,forgetting


In [ ]:
#Training_Aware
tasks = idx_tasks(5,ds_train,rand=False) 
task_train_dss = task_dss(tasks, ds_train,offset_change=False) 
task_test_dss = task_dss(tasks, ds_test,offset_change=False) 
task_train_dls = [DataLoader(ds,batch_size=16,shuffle=True,num_workers=4) for ds in task_train_dss]
task_test_dls = [DataLoader(ds, batch_size=16,shuffle=False,num_workers=4) for ds in task_test_dss]

trained_net_aware,df_table_aw_aw, forgetting= continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=False,agnsotic_test=False)
trained_net_aware,df_table_aw_agn, forgetting= continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=False,agnsotic_test=True)
print(df_table_aw_aw)
print(df_table_aw_agn) 

In [ ]:
#Training_Agnostic
tasks = idx_tasks(5,ds_train,rand=False) 
task_train_dss = task_dss(tasks, ds_train,offset_change=True) 
task_test_dss = task_dss(tasks, ds_test,offset_change=False) 
task_train_dls = [DataLoader(ds,batch_size=16,shuffle=True,num_workers=4) for ds in task_train_dss]
task_test_dls = [DataLoader(ds, batch_size=16,shuffle=False,num_workers=4) for ds in task_test_dss]

trained_net_agnostic,df_table_agn_agn,forgetting = continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=True,agnsotic_test=True)
trained_net_agnostic,df_table_agn_aw,forgetting = continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=True,agnsotic_test=False)

print(df_table_agn_agn) 
print(df_table_agn_aw)

In [ ]:
print('AWARE')
print(df_table_aw_aw.head())
print(df_table_aw_agn.head()) 
print('AGNOSTIC')
print(df_table_agn_agn.head()) 
print(df_table_agn_aw.head())

In [ ]:
def plot_accuracies(df_agn,df_awa):
  x = ['task 0', 'task 1', 'task 2', 'task 3', 'task 4']
  y_joint = [77.6,77.6,77.6,77.6,77.6]
  y_first_agnostic = df_agn.first_training
  y_first_aware = df_awa.first_training
  y_last_agnostic = df_agn.last_training
  y_last_aware = df_awa.last_training
  plt.plot(x,y_joint,label='joint train')
  plt.plot(x,y_first_agnostic,color='b',marker='o',label='First train/Aware Test')
  plt.plot(x,y_first_aware,color='r',marker='o',label='First train/Agnostic Test')
  plt.plot(x,y_last_agnostic,color='b',linestyle='--',marker='o',label='Last train/Aware Test')
  plt.plot(x,y_last_aware,color='r',linestyle='--',marker='o',label='Last train/Agnostic Test')
  plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
  plt.xlabel('TASKS')
  plt.ylabel('ACCURACIES')
  plt.title('AGNOSTIC TRAINING')

In [ ]:
plot_accuracies(df_table_aw_aw,df_table_aw_agn)

In [ ]:
plot_accuracies(df_table_agn_aw,df_table_agn_agn)

In [ ]:
# CLASSE IL DATASET FILTRATO
class Filtered_Dataset_Replay(Dataset): 
  def __init__(self,dataset,indices,tasks,idx,offset=0,replay_num=100):
        self.indices = indices # indici dei target di cui ci interessa il target
        self.original_indices = [i for i in range(len(dataset.targets)) if dataset.targets[i] in indices] # filtraggio degli indici del dataset che voglio nel subset
        if idx!=0:
          for id in range(0,idx):
           self.replay_indices = [i for i in range(len(dataset.targets)) if dataset.targets[i] in tasks[id]]
           self.replay_indices = random.sample(self.replay_indices,replay_num)
           self.original_indices = self.original_indices + self.replay_indices # concateno gli indici
           self.indices = np.concatenate((tasks[id], self.indices), axis=None)
           self.indices = np.arange(len(self.indices))
        self.dataset = Subset(dataset, self.original_indices)
        self.original2task = { self.indices[i]  : offset + i for i in range(0,  len(self.indices) ) } # remapping degli indici con chiavi i valori degli indici del del task
        #self.original2task = self.dictmap(self.indices,offset)
        self.task2original = dict({ (val, key) for (key, val) in self.original2task.items() }) # remapping degli indici con chiavi i valori degli indici del del task
        
  def __getitem__(self, idx):
        (x, y) = self.dataset[idx]
        return (x, self.original2task[y])

  def __len__(self):
        return len(self.original_indices)

  def dictmap(self,indices,offset):
      original2task={}
      idx = np.arange(len(indices))
      print(idx)
      idx_tmp = np.arange(len(indices))
      for i in range(0,len(idx)):
        if (idx[i]+offset) in indices:
          original2task[idx[i]+offset] = (idx[i] + offset)
          indices  = np.delete(indices, np.argwhere(indices == (idx[i]+offset)))
          idx_tmp  = np.delete(idx_tmp,np.argwhere(idx_tmp == (idx[i])))     
      for i in range(0,len(indices)):
          original2task[indices[i]]=idx_tmp[i] + offset   
      return original2task


def task_dss_replay(tasks,ds,offset_change=True): # aggiunto check su offset in modo tale da non cambaire offset ai task
    task_dss = []
    offset = 0
    for i in range(0,len(tasks)) :
      print(offset)
      tmp = Filtered_Dataset_Replay(ds, tasks[i], tasks, i, offset)
      task_dss.append(tmp)
      if offset_change:
          offset += len(tasks[i])
      else :
        offset = 0    
    return task_dss  

In [ ]:
tasks = idx_tasks(5,ds_train,rand=False) 
task_train_dss_replay = task_dss_replay(tasks, ds_train,offset_change=False) 
task_test_dss = task_dss(tasks, ds_test,offset_change=False) 
task_train_dls = [DataLoader(ds,batch_size=16,shuffle=True,num_workers=4) for ds in task_train_dss_replay]
task_test_dls = [DataLoader(ds, batch_size=16,shuffle=False,num_workers=4) for ds in task_test_dss]

0
0
0
0
0
0
0
0
0
0


In [ ]:
task_train_dss_replay[3].original2task
#task_train_dss_replay[3].indices

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}

In [ ]:
#Training_Agnostic
tasks = idx_tasks(5,ds_train,rand=False) 
task_train_dss_replay = task_dss_replay(tasks, ds_train,offset_change=False) 
task_test_dss = task_dss(tasks, ds_test,offset_change=False) 
task_train_dls = [DataLoader(ds,batch_size=16,shuffle=True,num_workers=4) for ds in task_train_dss_replay]
task_test_dls = [DataLoader(ds, batch_size=16,shuffle=False,num_workers=4) for ds in task_test_dss]

trained_net_agnostic,df_table_agn_agn,forgetting = continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=True,agnsotic_test=True)
trained_net_agnostic,df_table_agn_aw,forgetting = continual_table(tasks,task_train_dls,task_test_dls,task_test_dss,agnostic_train=True,agnsotic_test=False)

print(df_table_agn_agn) 
print(df_table_agn_aw)

0
0
0
0
0
0
0
0
0
0
Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=16384, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Tempo dell'epoca  0 uguale a:  5.743764379003551 sec
Tempo dell'epoca  1 uguale a:  5.652125218002766 sec
Tempo dell'epoca  2 uguale a:  5.6669350939992 sec
Tempo dell'epoca  3 uguale a:  5.647960206999414 sec
Tempo dell'epoca  4 uguale a:  5.68731149599